# IMPORTS AND KEYS

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

import json
import requests

I created two comicvine accounts so that I could rate API_KEYs when I reach the API rate limits.

In [2]:
API_KEY = "4ace0c77025132b5ae59c770af4ba01b80444161"
BASE_URL = "https://comicvine.gamespot.com/api/"

In [12]:
API_KEY = "f7ca7f049ce4f06afd6c5c1e082061cd34779ff9"
BASE_URL = "https://comicvine.gamespot.com/api/"

# API FUNCTIONS
The following four functions are what I use to extract the data I want from the ComicVine API. 

In [3]:
def fetch_volume_id(volume_name):
    API_URL = BASE_URL + 'volumes/'
    headers = {"User-Agent": "Mozilla/5.0"}
    params = {
        'api_key': API_KEY,
        'format': 'json',
        'filter': volume_name
    }
    try:
        response = requests.get(API_URL, headers=headers, params=params)
        response.raise_for_status()
        volumes = response.json().get('results', [])
        for volume in volumes:
            if volume_name.lower() in volume['name'].lower():
                return volume['id']
        return None
    except requests.exceptions.RequestException as e:
        print(f'Error fetching volumes: {e}')
        print(f'Response content: {response.content}')  # Print response content for debugging
        return None

In [4]:
# Function to fetch issues for a specific volume
def fetch_volume_issues(volume_id, offset=0):
    API_URL = BASE_URL + 'volume/4050-' + str(volume_id) + '/'
    headers = {"User-Agent": "Mozilla/5.0"}
    params = {
        'api_key': API_KEY,
        'format': 'json',
        'offset': offset,
        'limit': 100  # Adjust as needed, default ComicVine API limit per request
    }
    try:
        response = requests.get(API_URL, headers=headers, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f'Error fetching volume issues: {e}')
        print(f'Response content: {response.content}')  # Print response content for debugging
        return {}

In [5]:
# Function to fetch detailed information for a single issue
def fetch_issue(api_detail_url):
    headers = {"User-Agent": "Mozilla/5.0"}
    params = {
        'api_key': API_KEY,
        'format': 'json'
    }
    try:
        response = requests.get(api_detail_url, headers=headers, params=params)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f'Error fetching issue details: {e}')
        return {}

In [6]:
def combined_issues(volume_name):
    volume_id = fetch_volume_id(volume_name)
    if not volume_id:
        print(f'Volume "{volume_name}" not found.')
        return pd.DataFrame()

    all_issues = []
    offset = 0

    try:
        while True:
            response = fetch_volume_issues(volume_id, offset)
            issues = response.get('results', [])
            all_issues.extend(issues)

            if not issues:
                break
            
            if not response.get('number_of_page_results'):
                break
            
            offset += 100  # Adjust as needed based on the API limit
            time.sleep(3)  # To avoid hitting API rate limits

    except Exception as e:
        print(f'Error occurred: {e}', offset)
        return pd.DataFrame()

    table_data = []

    for issue in all_issues:
        if isinstance(issue, dict):
            detail_url = issue.get('api_detail_url')
            if detail_url:
                details = fetch_issue(detail_url).get('results', {})
                table_data.append(details)
                time.sleep(3)  # To avoid hitting API rate limits

    df = pd.DataFrame(table_data)
    return df

# ACUTALLY USING EVERYTHING

In [56]:
#fetch_volume_issues(volume_id, offset=0)
#I directly use the ids that I find on the ComicVine website. 
data = fetch_volume_issues(69322, 0)

In [57]:
data_prime = data['results']

In [58]:
issues_lst = data_prime.get('issues')

In [59]:
table_data = []
i, n = 0, 0 #73
for issue in issues_lst:
    #if (n >= 201): #only use when I want to start collecting data from a certain index (middle) in issues_lst
    url = issue.get('api_detail_url')
    details = fetch_issue(url).get('results', {})
    table_data.append(details)
    print(n, end=" ")
    #time.sleep(25) #use to try and stay within API rate limits
    n+=1

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 

In [60]:
df = pd.DataFrame(table_data)

In [35]:
tmp = pd.read_csv("data/detective_comics_vol1.csv")

In [15]:
df = pd.concat([df, tmp], ignore_index=True)

In [61]:
#export csv file to the data folder
df.to_csv('data/harley_quinn_vol2.csv')